# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, Sst, Male, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISl sessions. The Female SST VISp sessions have 0 low-contamination units. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('Sst') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
756029989,2019-10-03T00:00:00Z,734865738,brain_observatory_1.1,96.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,684,2214,6,"[TH, DG, CA3, CA1, VISl, nan, PO, Eth, LP, VIS..."
758798717,2019-10-03T00:00:00Z,735109609,brain_observatory_1.1,102.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,593,1486,4,"[MGv, MGd, TH, DG, CA1, VISal, nan, SGN, LP, S..."
762602078,2019-10-03T00:00:00Z,744915204,brain_observatory_1.1,110.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,531,2233,6,"[MB, APN, NOT, DG, CA1, VISam, nan, VISrl, LGv..."


In [7]:
session_id = filtered_sessions.index.values[4]

session = cache.get_session_data(session_id)

Downloading:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

/Users/tulsipatel/.local/lib/python3.8/site-packages/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_warehouse_api.py:291: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  pv_is_bool = np.issubdtype(output["p_value_rf"].values[0], np.bool)


Total number of units: 531


,waveform_spread,d_prime,waveform_halfwidth,waveform_amplitude,amplitude_cutoff,isolation_distance,waveform_repolarization_slope,nn_hit_rate,silhouette_score,max_drift,cumulative_drift,L_ratio,waveform_recovery_slope,presence_ratio,isi_violations,waveform_velocity_below,nn_miss_rate,waveform_duration,snr,waveform_PT_ratio,firing_rate,local_index_unit,peak_channel_id,waveform_velocity_above,cluster_id,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951859187,60.0,5.851167,0.082412,84.351930,0.073245,51.412508,0.543830,0.996667,NaN,26.53,349.67,0.000874,-0.215223,0.99,0.026998,NaN,0.000321,0.164824,3.976049,0.608781,12.235139,0,850059328,0.078488,0,NaN,300.0,1.319380,0.519947,0.726897,0.860606,0.494957,0.230309,0.006869,0.018369,0.023135,20.000,1.111881,0.695,0.04,8.0,NaN,NaN,-0.288961,NaN,NaN,0.0,150.0,NaN,NaN,0.191648,NaN,NaN,-30.000,4969,0.00,12.646553,13.455496,11.882904,11.014312,12.016615,NaN,0.0445,False,False,False,False,False,-0.092339,False,0.003157,0.000071,0.004938,0.013786,0.005589,20,11,768909178,1,313.0,MB,8356.0,3127.0,6956.0,probeA,See electrode locations,29999.965191,1249.99855,True
951859208,30.0,4.455170,0.219765,76.825320,0.005747,469.518069,0.192688,0.999333,0.122151,78.78,669.23,0.000264,-0.105245,0.99,0.005189,NaN,0.015839,0.109883,2.117930,1.534792,59.022372,2,850059330,0.343384,2,0.458235,600.0,33.036501,0.773333,1.543836,1.366667,1.284690,0.188514,0.017478,0.101261,0.055970,81.667,0.802367,0.382,0.04,15.0,NaN,NaN,NaN,NaN,NaN,0.0,30.0,NaN,NaN,NaN,NaN,NaN,3.333,4958,0.75,46.815119,2.051630,96.250112,1.362946,88.037022,NaN,0.0335,False,False,False,False,False,0.006966,False,0.063286,0.016393,0.003063,0.153659,0.007239,40,59,768909178,2,313.0,MB,8352.0,3119.0,6958.0,probeA,See electrode locations,29999.965191,1249.99855,True
951861908,60.0,3.855213,0.096147,131.692665,0.047835,34.093637,0.544336,0.546296,-0.215104,45.82,955.56,0.010626,-0.163178,0.99,0.052207,0.000000,0.000672,0.178559,3.362200,0.613861,0.838901,284,850059336,-0.206030,287,1.065773,300.0,10.203030,0.846667,1.600000,0.566667,1.030733,1.477649,0.099507,0.057778,0.154232,13.333,0.874737,0.193,0.32,2.0,-0.791667,NaN,NaN,NaN,-0.863636,135.0,30.0,0.487679,NaN,NaN,NaN,0.272816,-26.667,4975,0.00,0.868378,1.172360,0.338533,2.628303,0.353704,NaN,0.0115,False,False,False,False,False,0.152542,True,0.082968,0.016393,0.190495,0.096522,0.221537,60,11,768909178,5,313.0,MB,8342.0,3095.0,6966.0,probeA,See electrode locations,29999.965191,1249.99855,True
951859229,70.0,5.878799,0.082412,139.016670,0.010283,61.117828,0.483340,0.972000,0.139767,36.11,466.24,0.002311,-0.194041,0.99,0.129645,0.000000,0.003653,0.151089,3.363163,0.627144,7.751163,4,850059336,-0.156975,4,1.044380,900.0,28.756491,1.702342,5.691304,2.070440,11.095072,0.787597,0.077434,0.029862,0.175562,25.556,0.937044,0.422,0.16,2.0,NaN,NaN,-0.606061,NaN,NaN,135.0,90.0,NaN,NaN,0.008209,NaN,NaN,18.889,4924,0.25,9.425611,7.407184,6.401897,7.485785,6.332703,NaN,0.0885,False,False,False,Fal

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 47


,waveform_spread,d_prime,waveform_halfwidth,waveform_amplitude,amplitude_cutoff,isolation_distance,waveform_repolarization_slope,nn_hit_rate,silhouette_score,max_drift,cumulative_drift,L_ratio,waveform_recovery_slope,presence_ratio,isi_violations,waveform_velocity_below,nn_miss_rate,waveform_duration,snr,waveform_PT_ratio,firing_rate,local_index_unit,peak_channel_id,waveform_velocity_above,cluster_id,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951844342,60.0,4.462594,0.219765,55.821675,0.004954,60.118534,0.174724,0.977987,0.110046,32.17,246.95,0.003146,-0.020808,0.99,0.016876,-0.549414,0.000585,0.727973,1.960029,0.320375,2.086661,197,850064204,0.686767,200,0.235757,800.0,1.461708,0.472864,0.811154,0.776190,0.900219,0.552538,0.066288,0.142829,0.194632,22.50,0.357420,0.0,0.04,1.0,NaN,NaN,0.295918,NaN,NaN,0.0,0.0,NaN,NaN,2.642985e-01,NaN,NaN,41.25,5023,0.50,2.675558,4.849308,2.758639,1.937510,1.445458,NaN,0.0975,False,False,False,False,False,0.723746,False,0.179165,0.478037,0.341474,0.305933,0.274619,2080,59,768909182,206,385.0,VISp,8908.0,1527.0,8029.0,probeC,See electrode locations,29999.995408,1249.999809,True
951846069,90.0,2.822695,0.206030,71.065410,0.044939,47.689747,0.225363,0.749333,0.042964,41.24,233.02,0.028895,-0.012493,0.99,0.068459,-0.755444,0.010317,0.837856,2.305030,0.348285,2.930363,411,850064212,1.648241,420,0.073176,800.0,1.983333,0.967619,0.591828,0.494872,0.614403,0.365422,0.063716,0.697554,0.758744,38.75,4.163816,0.0,0.04,1.0,NaN,NaN,0.472527,NaN,0.917808,135.0,120.0,NaN,NaN,7.740336e-11,NaN,0.024598,38.75,5003,0.75,3.426917,2.158208,2.116502,3.009884,3.635625,NaN,0.0935,False,False,False,False,False,0.722441,False,0.669344,0.255517,0.612353,0.390884,0.714237,2120,59,768909182,210,385.0,VISp,8919.0,1493.0,8037.0,probeC,See electrode locations,29999.995408,1249.999809,True
951844385,70.0,2.555699,0.219765,78.060450,0.016737,77.522686,0.216142,0.926667,0.074004,37.80,148.99,0.010719,-0.008663,0.99,0.048747,-0.412060,0.027738,0.810385,2.111120,0.437185,8.236168,203,850064212,0.343384,206,NaN,800.0,1.828401,0.804903,1.694286,0.490043,1.628122,0.474450,0.128344,0.506660,0.200218,31.25,1.334192,0.0,0.32,2.0,NaN,NaN,NaN,NaN,0.293722,315.0,150.0,NaN,NaN,NaN,NaN,0.503117,18.75,5017,0.00,9.591873,15.613705,5.423913,8.928775,6.090239,NaN,0.2305,False,False,False,False,False,0.397695,False,0.389885,0.223783,0.220893,0.118298,0.392115,2120,59,768909182,210,385.0,VISp,8919.0,1493.0,8037.0,probeC,See electrode locations,29999.995408,1249.999809,True
951844377,60.0,3.582919,0.219765,88.802220,0.004089,71.013899,0.254756,0.941333,0.129274,42.00,97.35,0.019752,-0.021601,0.99,0.031626,-0.824121,0.007939,0.837856,2.580469,0.380204,8.208525,202,850064212,6.180905,205,NaN,600.0,1.278569,1.148776,0.381755,0.573913,0.572093,0.284246,0.097359,0.656969,0.400960,50.00,3.243948,0.0,0.02,1.0,NaN,NaN,NaN,NaN,NaN,270.0,120.0,NaN,NaN,NaN,NaN,NaN,45.00,4933,0.50,7.509883,23.234045,9.657588,5.819109,11.409123,NaN,0.0985,Fal

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
951844342     1.445458
951846069     3.635625
951844385     6.090239
951844377    11.409123
951844370     0.927890
951844425     0.538883
951844476     8.758673
951844470     1.053785
951844456    15.401117
951844568     3.982002
951844520    39.586219
951844591     7.435780
951844582     5.864428
951844634     3.002821
951844746     0.350374
951844737    11.400463
951844858     8.497558
951844851    37.835683
951846133     4.466930
951844879    11.860745
951844872     6.178831
951844911    10.304713
951844903    17.613932
951844886    11.593635
951846428     5.345528
951844956    14.889545
951844933     0.388342
951844926     0.399666
951844918    12.469569
951845004     4.204483
951844995     1.163693
951844981    15.837418
951844975    14.041587
951844894    11.792802
951845018     9.493392
951846148    10.768325
951845065     9.185649
951846190     1.300246
951845108     5.201649
951845098     7.930699
951845153     3.601654
951845135     9.957670
951845126     3.196659
951

In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

8.459462497523758


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
951844342     2.675558
951846069     3.426917
951844385     9.591873
951844377     7.509883
951844370     5.413068
951844425     0.528440
951844476     3.086979
951844470     0.746594
951844456    14.318181
951844568    10.409420
951844520    26.515718
951844591     7.951486
951844582     0.937213
951844634     3.934707
951844746     0.182677
951844737     7.651260
951844858     4.269879
951844851    24.610581
951846133     2.904302
951844879     6.628797
951844872     5.912385
951844911     6.506483
951844903    10.218271
951844886     4.503918
951846428     9.967288
951844956     7.434165
951844933     0.284870
951844926     0.577683
951844918     8.630833
951845004     2.267844
951844995     0.203328
951844981     7.479172
951844975     8.139987
951844894     0.852493
951845018     4.748546
951846148     5.857317
951845065     5.346351
951846190     0.195915
951845108     1.749994
951845098     4.194691
951845153    10.586272
951845135     1.333278
951845126     1.430706
951

In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

5.441964764497123


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])